<a href="https://colab.research.google.com/github/Wyatthy/NLP/blob/main/NLPtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [2]:
#import
#from snownlp import SnowNLP
from tensorflow import keras
from gensim.models.word2vec import Word2Vec
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout, Activation
from sklearn.model_selection import train_test_split
from keras.models import load_model
import csv
import imp
import numpy as np
import yaml
import sys
import jieba
import re
import gensim
import multiprocessing
# import keras.utils
# from google.colab import drive
# drive.mount('/content/drive')

# infer

In [3]:
#infer 
voc_dim = 150 
model_word=Word2Vec.load('/content/drive/MyDrive/NLP/model/Word2Vec_java.pkl')

input_dim = len(model_word.wv.vocab.keys()) + 1 
embedding_weights = np.zeros((input_dim, voc_dim)) 
w2dic={} 
for i in range(len(model_word.wv.vocab.keys())): 
  embedding_weights[i + 1, :] = model_word.wv[list(model_word.wv.vocab.keys())[i]] 
  w2dic[list(model_word.wv.vocab.keys())[i]]=i+1 
  #print(list(model_word.wv.vocab.keys())[i])

model = load_model('/content/drive/MyDrive/NLP/model/lstm_java_total.h5')

pchinese = re.compile('([\u4e00-\u9fa5]+)+?')

label={0:"happy",1:"angry",2:"disgust",3:"sad"} 
#in_stc=["明天","就要","考试","我","特别","紧张","一点","都","没有","复习"] 
in_str="抑郁"

in_stc=''.join(pchinese.findall(in_str))

in_stc=list(jieba.cut(in_stc,cut_all=True, HMM=False)) 
# s=SnowNLP(in_stc) 
# in_stc=s.words 
new_txt=[]
data=[] 
for word in in_stc: 
  try: 
    new_txt.append(w2dic[word]) 
  except: 
    new_txt.append(0) 
    data.append(new_txt)

data=sequence.pad_sequences(data, maxlen=voc_dim ) 
pre=model.predict(data)[0].tolist()

print(pre) 
print("输入:\n "+in_str) 
print("输出:\n "+label[pre.index(max(pre))])

KeyboardInterrupt: ignored

# DealCSV

In [4]:
#dealCSV 处理数据集

sourcefile = '/content/drive/MyDrive/NLP/data/simplifyweibo_4_moods.csv'
happyfile = '/content/drive/MyDrive/NLP/data/happy.txt'
angryfile = '/content/drive/MyDrive/NLP/data/angry.txt'
disgustfile = '/content/drive/MyDrive/NLP/data/disgust.txt'
sadfile = '/content/drive/MyDrive/NLP/data/sad.txt'
a=b=c=d=0

with open(sourcefile, encoding='UTF-8') as f, open(happyfile, 'w', encoding='utf-8') as fp1, open(angryfile, 'w', encoding='utf-8') as fp2, open(disgustfile, 'w', encoding='utf-8') as fp3, open(sadfile, 'w', encoding='utf-8') as fp4:
    reader = csv.reader(f)
    head_row = next(reader)
    for row in reader:
        if(row[0]=='0' and a<10000):
            fp1.write(row[1]+'\n')
            a+=1
        elif(row[0]=='1' and b<10000):
            fp2.write(row[1]+'\n')
            b+=1
        elif(row[0]=='2' and c<10000):
            fp3.write(row[1]+'\n')
            c+=1
        elif(row[0]=='3' and d<10000):
            fp4.write(row[1]+'\n')
            d+=1

# dataset

In [5]:
#dataset
#s = SnowNLP(u'这个东西真心很赞')

def clean_data(rpath,wpath):
    pchinese = re.compile('([\u4e00-\u9fa5]+)+?') #搞了一个正则表达对象 避免重复转换
    f = open(rpath,encoding="UTF-8")
    fw = open(wpath, "w",encoding="UTF-8")
    for line in f.readlines():
        m = pchinese.findall(str(line))
        if m:
            str1 = ''.join(m)
            str2 = str(str1)
            fw.write(str2)
            fw.write("\n")
    f.close()
    fw.close()

def seg_char(sent):  #分字
    """
    把句子按字分开，不破坏英文结构
    """
    # 首先分割 英文 以及英文和标点
    pattern_char_1 = re.compile(r'([\W])')
    parts = pattern_char_1.split(sent)
    parts = [p for p in parts if len(p.strip())>0]
    # 分割中文
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = pattern.split(sent)
    chars = [w for w in chars if len(w.strip())>0]
    return chars

def loadfile():
    happy = []
    angry = []
    disgust=[]
    sad=[]
    with open('/content/drive/MyDrive/NLP/data/happy_clean.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            #happy.append(seg_char((line)))
            happy.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
    with open('/content/drive/MyDrive/NLP/data/angry_clean.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            #angry.append(seg_char((line)))
            angry.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('/content/drive/MyDrive/NLP/data/disgust_clean.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            #disgust.append(seg_char((line)))
            disgust.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()
    with open('/content/drive/MyDrive/NLP/data/sad_clean.txt',encoding='UTF-8') as f:
        for line in f.readlines():
            #sad.append(seg_char((line)))
            sad.append(list(jieba.cut(line, cut_all=False, HMM=True))[:-1])
        f.close()

    X_Vec = np.concatenate((happy,angry,disgust,sad))

    y = np.concatenate((np.zeros(len(happy), dtype=int),
                        np.ones(len(angry), dtype=int),
                        2*np.ones(len(disgust), dtype=int),
                        3*np.ones(len(sad), dtype=int)))

    return X_Vec, y

# Word2Vec

In [6]:
#Word2Vec
voc_dim = 150 #word的向量维度
min_out = 4  #单词出现频率数
window_size = 7 #
cpu_count = multiprocessing.cpu_count()

def word2vec_train(X_Vec):
    model_word = Word2Vec(size=voc_dim,#vector_size=voc_dim
                min_count=min_out,
                window=window_size,
                workers=cpu_count,
                iter=100) # an empty model, no training yet
    model_word.build_vocab(X_Vec)
    model_word.train(X_Vec, total_examples=model_word.corpus_count, epochs=model_word.epochs)
    model_word.save('/content/drive/MyDrive/NLP/model/Word2Vec_java.pkl')

    print(len(model_word.wv.vocab))
    input_dim = len(model_word.wv.vocab.keys()) + 1 #频数小于阈值的词语统统放一起，编码为0
    embedding_weights = np.zeros((input_dim, voc_dim))
    w2dic={}
    for i in range(len(model_word.wv.vocab.keys())):
        #embedding_weights[i+1, :] = model_word[list(model_word.wv.vocab.keys())[i]]
        embedding_weights[i + 1, :] = model_word.wv[list(model_word.wv.vocab.keys())[i]]
        w2dic[list(model_word.wv.vocab.keys())[i]]=i+1
    return input_dim,embedding_weights,w2dic

# LSTM

In [7]:
#lstm
lstm_input = 150#lstm输入维度
voc_dim = 150 #word的向量维度
def lstm(input_dim, embedding_weights):
    model = Sequential()
    model.add(Embedding(output_dim=voc_dim,
                        input_dim=input_dim,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=lstm_input))
    model.add(LSTM(256, activation='softsign'))
    model.add(Dropout(0.5))
    model.add(Dense(4))
    model.add(Activation('softmax'))
    return model

# main

In [8]:
#Main
imp.reload(sys)
np.random.seed()
#参数

voc_dim = 150 #word的向量维度
lstm_input = 150#lstm输入维度
epoch_time = 10#epoch
batch_size = 32 #batch

def data2inx(w2indx,X_Vec):
    data = []
    for sentence in X_Vec:
        new_txt = []
        for word in sentence:
            try:
                new_txt.append(w2indx[word])
            except:
                new_txt.append(0)

        data.append(new_txt)
    return data
def train_lstm(model, x_train, y_train, x_test, y_test):
    print('Compiling the Model...')
    model.compile(loss='binary_crossentropy',#hinge
                  optimizer='adam', metrics=['mae', 'acc'])

    print("Train..." )
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch_time, verbose=1)

    print("Evaluate...")
    print(model.predict(x_test))
    score = model.evaluate(x_test, y_test,
                           batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('/content/drive/MyDrive/NLP/model/lstm_java.yml', 'w') as outfile:
        outfile.write(yaml.dump(yaml_string, default_flow_style=True))
    model.save('/content/drive/MyDrive/NLP/model/lstm_java_total.h5')
    print('Test score:', score)

print("开始清洗数据................")
clean_data('/content/drive/MyDrive/NLP/data/happy.txt','/content/drive/MyDrive/NLP/data/happy_clean.txt')
clean_data('/content/drive/MyDrive/NLP/data/angry.txt','/content/drive/MyDrive/NLP/data/angry_clean.txt')
clean_data('/content/drive/MyDrive/NLP/data/disgust.txt','/content/drive/MyDrive/NLP/data/disgust_clean.txt')
clean_data('/content/drive/MyDrive/NLP/data/sad.txt','/content/drive/MyDrive/NLP/data/sad_clean.txt')
print("清洗数据完成................")
print("开始下载数据................")
X_Vec, y=loadfile()
print("下载数据完成................")
print("开始构建词向量................")
input_dim,embedding_weights,w2dic = word2vec_train(X_Vec)
print("构建词向量完成................")

index = data2inx(w2dic,X_Vec)
index2 = sequence.pad_sequences(index, maxlen=voc_dim )
x_train, x_test, y_train, y_test = train_test_split(index2, y, test_size=0.2)
y_train = keras.utils.to_categorical(y_train, num_classes=4)
y_test = keras.utils.to_categorical(y_test, num_classes=4)

model=lstm(input_dim, embedding_weights)
train_lstm(model, x_train, y_train, x_test, y_test)

开始清洗数据................


Building prefix dict from the default dictionary ...


清洗数据完成................
开始下载数据................


Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.017 seconds.
Prefix dict has been built successfully.
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


下载数据完成................
开始构建词向量................
26913
构建词向量完成................
Compiling the Model...
Train...
Epoch 1/10
998/998 [==============================] - 700s 698ms/step - loss: 0.5592 - mae: 0.3585 - acc: 0.3235
Epoch 2/10
998/998 [==============================] - 695s 697ms/step - loss: 0.5003 - mae: 0.3183 - acc: 0.4354
Epoch 3/10
998/998 [==============================] - 693s 694ms/step - loss: 0.4454 - mae: 0.2778 - acc: 0.5108
Epoch 4/10
998/998 [==============================] - 681s 683ms/step - loss: 0.3766 - mae: 0.2321 - acc: 0.5846
Epoch 5/10
998/998 [==============================] - 684s 685ms/step - loss: 0.3129 - mae: 0.1941 - acc: 0.6497
Epoch 6/10
998/998 [==============================] - 693s 694ms/step - loss: 0.2670 - mae: 0.1688 - acc: 0.6927
Epoch 7/10
998/998 [==============================] - 678s 679ms/step - loss: 0.2379 - mae: 0.1533 - acc: 0.7126
Epoch 8/10
998/998 [==============================] - 682s 683ms/step - loss: 0.2209 - mae: 0.1454 -